# Wordle helper 

Simple Julia code to help with playing the game Wordle. 

## Loading the code

In [2]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

import DataStructures: counter
import Random: shuffle 

include(joinpath(@__DIR__, "wordle_helper.jl"));

  Activating project at `~/Github/Wordle_helper`


## Finding the best starting word

In [3]:
let 
    all_words = load_word_lists()
    find_best_guess(; all_words..., verbose = true)
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:08:44


"roate"

Using an entropy measure for the score:

In [4]:
let 
    all_words = load_word_lists()
    find_best_guess(; all_words..., verbose = true, use_entropy = true)
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:08:43


"soare"

## Solving some words

Testing some solutions

In [5]:
let 
    all_words = load_word_lists()
    for w in ["epoxy", "lodge", "wound", "awash", "raise"] 
        println(solve(w, "raise"; all_words...))
    end 
end

["raise", "denet", "epoch", "epoxy"]
["raise", "nould", "lodge"]
["raise", "could", "bumph", "found", "wound"]
["raise", "slash", "about", "awash"]
["raise"]


In [6]:
let 
    all_words = load_word_lists()
    for w in ["epoxy", "lodge", "wound", "awash", "raise"] 
        println(solve(w, "raise"; all_words..., use_entropy = true))
    end 
end

["raise", "denet", "epoch", "epoxy"]
["raise", "nould", "lodge"]
["raise", "mulch", "bawty", "wound"]
["raise", "slash", "about", "awash"]
["raise"]


### Hard mode

In [7]:
let 
    all_words = load_word_lists()
    for w in ["epoxy", "lodge", "wound", "awash", "raise"] 
        println(solve(w, "roate"; all_words..., hard_mode = true))
    end
end  

["roate", "demon", "epoch", "epoxy"]
["roate", "louse", "lodge"]
["roate", "goldy", "dowps", "wound"]
["roate", "slack", "unais", "awash"]
["roate", "raise"]


## Solving all games

In [8]:
function solve_all_words(; hard_mode = false, first_guess = "roate", use_entropy = false)
    all_words = load_word_lists()
    p = Progress(length(all_words.words))
    sols = similar(all_words.words, Any)
    @floop for (i, word) in pairs(shuffle(all_words.words))
        next!(p)
        sols[i] = solve(word, first_guess; all_words..., hard_mode, use_entropy)
    end 
    return sols
end

solve_all_words (generic function with 1 method)

In [9]:
let 
    sol = solve_all_words()
    counter(map(x -> length(x), sol)) |> collect |> sort
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:15:27


4-element Vector{Pair{Int64, Int64}}:
 2 => 54
 3 => 1126
 4 => 1095
 5 => 40

This algorithm breaks all Wordle puzzles under 5 guesses. 

Using the entropy score:

In [10]:
let 
    sol = solve_all_words(use_entropy = true)
    counter(map(x -> length(x), sol)) |> collect |> sort
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:15:21


4-element Vector{Pair{Int64, Int64}}:
 2 => 58
 3 => 1145
 4 => 1067
 5 => 45

### Hard mode

In [11]:
let 
    sol = solve_all_words(; hard_mode = true)
    println(counter(map(x -> length(x), sol)) |> collect |> sort)
    map(last, sol[findall(x -> length(x) > 6, sol)])
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:00:57


[2 => 92, 3 => 1010, 4 => 986, 5 => 184, 6 => 35, 7 => 7, 8 => 1]


8-element Vector{String}:
 "joker"
 "patch"
 "watch"
 "vaunt"
 "sower"
 "willy"
 "wight"
 "waste"

Using the entropy score:

In [12]:
let 
    sol = solve_all_words(; hard_mode = true, use_entropy = true)
    counter(map(x -> length(x), sol)) |> collect |> sort |> println
    map(last, sol[findall(x -> length(x) > 6, sol)])
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:00:58


[2 => 99, 3 => 1038, 4 => 965, 5 => 177, 6 => 30, 7 => 5, 8 => 1]


6-element Vector{String}:
 "waste"
 "wight"
 "willy"
 "vaunt"
 "watch"
 "match"

In [13]:
let 
    sol = solve_all_words(; first_guess = "soare", hard_mode = true, use_entropy = true)
    counter(map(x -> length(x), sol)) |> collect |> sort |> println
    map(last, sol[findall(x -> length(x) > 6, sol)])
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:00:42


[2 => 92, 3 => 1033, 4 => 951, 5 => 189, 6 => 35, 7 => 11, 8 => 4]


15-element Vector{String}:
 "swore"
 "jaunt"
 "jolly"
 "tower"
 "vaunt"
 "patch"
 "waste"
 "shave"
 "rower"
 "wound"
 "willy"
 "watch"
 "rover"
 "tried"
 "pound"

## Breaking one Wordle

### Wordle 220

In [14]:
all_words = load_word_lists()
game_words = load_word_lists().words

new_guess_and_update!(game_words, "roate", [1, 0, 1, 0, 0], all_words.allowed_guesses)

"carrs"

In [15]:
new_guess_and_update!(game_words, "carrs", [0, 1, 1, 0, 1], all_words.words)

"sugar"

### Wordle 223 

In [16]:
all_words = load_word_lists()
game_words = load_word_lists().words

new_guess_and_update!(game_words, "roate", [1, 0, 0, 0, 1], all_words.words)

"spied"

In [17]:
new_guess_and_update!(game_words, "spied", [0, 1, 0, 1, 0], all_words.words)

"perch"

In [18]:
new_guess_and_update!(game_words, "perch", [2, 2, 2, 0, 0], all_words.words)

"perky"

### Hard mode. 


Wordle 222 

In [19]:
all_words = load_word_lists()
game_words = load_word_lists().words

new_guess_and_update!(game_words, "roate", [0, 2, 0, 1, 0], game_words, hard_mode = true)

"joint"

In [20]:
new_guess_and_update!(game_words, "joint", [0, 2, 0, 2, 2], game_words, hard_mode = true)

"count"

In [21]:
new_guess_and_update!(game_words, "count", [0, 2, 2, 2, 2], game_words, hard_mode = true)

"mount"